In [10]:
# !wget https://raw.githubusercontent.com/huseinzol05/Pyspark-ML/master/notebooks/all_book_titles.txt
# !pip3 install nltk

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StringType, ArrayType
from pyspark.ml.feature import Word2Vec
from operator import add
from nltk.corpus import stopwords

In [14]:
stopwords = stopwords.words('english')

In [4]:
sess = SparkSession.builder.appName('nlp').getOrCreate()

22/02/10 14:59:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
lines = sess.sparkContext.textFile('all_book_titles.txt')

In [15]:
lines = lines \
    .map(lambda line: line.strip().lower()) \
    .map(lambda line: line.split()) \
    .map(lambda words: [w for w in words if w.isalpha()]) \
    .map(lambda words: [w for w in words if len(w) > 3]) \
    .map(lambda words: [w for w in words if w not in stopwords]).reduce(add)

In [17]:
df = sess.createDataFrame([(lines,), (lines,)], ['sentence'])

In [18]:
word2Vec = Word2Vec(vectorSize=64, inputCol='sentence', outputCol='model')

In [19]:
model = word2Vec.fit(df)

22/02/10 15:01:46 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/02/10 15:01:46 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [20]:
model.getVectors().show()

+------------+--------------------+
|        word|              vector|
+------------+--------------------+
|    embedded|[-0.0281618312001...|
|    feminism|[-0.0459480397403...|
|       unity|[0.08734160661697...|
|  conceptual|[0.04000809043645...|
|   reference|[0.01763781718909...|
|    workbook|[0.08311362564563...|
|     writing|[0.08494076132774...|
|    elements|[-0.0120139205828...|
|    discrete|[-0.0159764047712...|
|    semester|[0.16084235906600...|
|     measure|[-0.0345099978148...|
|      health|[0.04372465983033...|
| application|[-0.0353720448911...|
|     statics|[0.04680669680237...|
|  perceptive|[0.00351415248587...|
|organization|[-0.1775018423795...|
|    database|[0.04597432538866...|
|       moral|[-0.0295252725481...|
|     nursing|[0.09200063347816...|
|    politics|[-0.0211179312318...|
+------------+--------------------+
only showing top 20 rows



In [21]:
from pyspark.sql.functions import format_number as fmt

In [23]:
model.findSynonyms("politics", 20).select("word", fmt("similarity", 5).alias("similarity")).show()

+---------------+----------+
|           word|similarity|
+---------------+----------+
|        problem|   0.96499|
|    illustrated|   0.96113|
|          islam|   0.95739|
| bioinformatics|   0.94640|
|       circuits|   0.94578|
|pathophysiology|   0.93226|
|      actuarial|   0.93020|
|         primer|   0.92894|
|        judaism|   0.92451|
|     multimedia|   0.91843|
|      reduction|   0.91617|
| constitutional|   0.91257|
|       practice|   0.91221|
|       hardware|   0.90916|
|      computing|   0.90649|
|        century|   0.90640|
|        matters|   0.90288|
|        general|   0.89806|
|           drag|   0.89795|
|    linguistics|   0.89717|
+---------------+----------+

